In [11]:
import pandas as pd
import pickle
import seaborn as sns
import xgboost as xgb

from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error

In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={'data-scientist': 'soumyadip', 'model': 'xgboost'}>

In [3]:
def read_dataframe(filename):
    if filename.endswith(".csv"):
        df = pd.read_csv(filename)
        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    elif filename.endswith(".parquet"):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [6]:
len(df_train), len(df_val)

(73908, 61921)

In [7]:
df_train["PO_DO"] = df_train['PULocationID'] + "_" + df_train["DOLocationID"]
df_val["PO_DO"] = df_val['PULocationID'] + "_" + df_val["DOLocationID"]

In [8]:
categorical = ["PO_DO"]
numerical = ["trip_distance"]

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [12]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [17]:
from datetime import date
from tkinter.messagebox import ABORTRETRYIGNORE


with mlflow.start_run():

    tags = {
        "data-scientist": "Soumyadip",
        "model": "xgboost",
        "version": "v2",
    }    

    mlflow.set_tags(tags)
    
    best_params = {
        'learning_rate': 0.6383041146845452,
        'max_depth': 31,
        'min_child_weight': 1.1819369158769895,
        'objective': 'reg:linear',
        'reg_alpha': 0.14202144046856877,
        'reg_lambda': 0.18611039063872484,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50 
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    
    mlflow.xgboost.log_model(booster, artifact_path="model_mlflow")

[23:52:53] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:10.41313
[1]	validation-rmse:7.49404
[2]	validation-rmse:6.80830
[3]	validation-rmse:6.61693
[4]	validation-rmse:6.55181
[5]	validation-rmse:6.53427
[6]	validation-rmse:6.52150
[7]	validation-rmse:6.51310
[8]	validation-rmse:6.50617
[9]	validation-rmse:6.49763
[10]	validation-rmse:6.49332
[11]	validation-rmse:6.48786
[12]	validation-rmse:6.48156
[13]	validation-rmse:6.47422
[14]	validation-rmse:6.46948
[15]	validation-rmse:6.46388
[16]	validation-rmse:6.46056
[17]	validation-rmse:6.45686
[18]	validation-rmse:6.45375
[19]	validation-rmse:6.44925
[20]	validation-rmse:6.44596
[21]	validation-rmse:6.44087
[22]	validation-rmse:6.43587
[23]	validation-rmse:6.43250
[24]	validation-rmse:6.43028
[25]	validation-rmse:6.42795
[26]	validation-rmse:6.42820
[27]	validation-rmse:6.42383
[28]	validation-rmse

In [20]:
logged_model = 'runs:/104ad5c200e64ac6aa3eb01aacd43adb/model_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

[23:55:55] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [21]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model_mlflow
  flavor: mlflow.xgboost
  run_id: 104ad5c200e64ac6aa3eb01aacd43adb

In [22]:
xgboost_model = mlflow.xgboost.load_model(logged_model)


[23:58:12] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [30]:
# load and predict model
xgboost_model.predict(valid)[:10]

array([14.759042 ,  7.2535048, 17.462654 , 24.372429 ,  9.365672 ,
       17.207298 , 10.600252 ,  8.094475 ,  9.014834 , 20.498537 ],
      dtype=float32)